In [1]:
import os 

In [4]:
os.chdir(os.path.pardir)

In [6]:
# os.getcwd()

In [7]:
import pandas as pd 

In [71]:
gwinnett_pm25= pd.read_csv('data/external/fire_station_pm25_2011_2018.csv')

In [72]:
gwinnett_pm25['Date'] = pd.to_datetime(gwinnett_pm25['Date'])

In [73]:
gwinnett_pm25 = gwinnett_pm25.sort_values('Date')


In [74]:
gwinnett_pm25.index = gwinnett_pm25['Date']

In [75]:
import numpy as np

In [76]:
gwinnett_pm25 = gwinnett_pm25.asfreq('D', fill_value=np.nan)
gwinnett_pm25['Date'] = gwinnett_pm25.index

In [78]:
gwinnett_pm25 = gwinnett_pm25.reset_index(drop=True)

In [79]:
train_index = gwinnett_pm25[(gwinnett_pm25['Date'] >= '2011-01-01')
                                 & (gwinnett_pm25['Date'] <= '2016-12-31')].index.values
validation_index = gwinnett_pm25[(gwinnett_pm25['Date'] >= '2017-01-01')
                                 & (gwinnett_pm25['Date'] <= '2017-12-31')].index.values
test_index = gwinnett_pm25[(gwinnett_pm25['Date'] >= '2018-01-01')
                                 & (gwinnett_pm25['Date'] <= '2018-12-31')].index.values

In [117]:
# gwinnett_pm25[['Daily_Mean_PM2_5_Concentration']].describe()

,Daily_Mean_PM2_5_Concentration
count,906.000000
mean,10.466004
std,4.772114
min,0.300000
25%,7.000000
50%,10.000000
75%,13.100000
max,53.200000


In [152]:
gwinnett_pm25.iloc[590:605,]

,Date,Source,Site_ID,POC,Daily_Mean_PM2_5_Concentration,UNITS,DAILY_AQI_VALUE,Site_Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
590,2012-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
591,2012-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
592,2012-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593,2012-10-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,2012-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
595,2012-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,2012-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,2012-10-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,2012-10-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,2012-10-14,AQS,131210039.0,1.0,9.7,ug/m3 LC,40.0,Fire Station #8,1.0,100.0,88101.0,PM2.5 - Local Conditions,12060.0,"Atlanta-Sandy Springs-Roswell, GA",13.0,Georgia,121.0,Fulton,33.802241,-84.435618


In [187]:
pm25_matrix = np.array(gwinnett_pm25[['Daily_Mean_PM2_5_Concentration']])

In [92]:
pm25_matrix.shape

(2869, 1)

In [93]:
time_len = pm25_matrix.shape[0]

In [96]:
pol_sequences, pol_values = [], []

In [97]:
seq_len = 7
pred_len = 1

2139

365

365

In [119]:
pm25_matrix.shape

(2869, 1)

In [132]:
masking = (~np.isnan(pm25_matrix)).astype(int)

In [133]:
delta = np.zeros((masking.shape[0], masking.shape[1]))
for index, value in np.ndenumerate(masking):
    '''
    index: tuple of shape (row_num, col_num)
    value: numeric value
    '''
    
    # if first sample/day, delta equals 0 
    if index[0] == 0:
        delta[index[0], index[1]] = 0
    # if previous day is not observed 
    elif masking[index[0] - 1, index[1]] == 0:
        delta[index[0], index[1]] = 1 + delta[index[0] - 1, index[1]]
    # if previous day is observed 
    else:
        delta[index[0], index[1]] = 1

In [140]:
pm25_matrix.shape

(2869, 1)

In [141]:
masking.shape

(2869, 1)

In [143]:
delta.shape

(2869, 1)

In [153]:
# unique, counts = np.unique(delta, return_counts=True)
# dict(zip(unique, counts))

In [157]:
def get_feature_sequence(fea_matrix, seq_len=7):
    fea_sequences = []
    time_len = fea_matrix.shape[0]
    for i in range(time_len - seq_len - 1):
        fea_sequences.append(fea_matrix[i:i+seq_len])        
    return np.array(fea_sequences)

def get_label_sequence(label_matrix, seq_len=7):
    label_sequence = []
    time_len = fea_matrix.shape[0]
    for i in range(time_len - seq_len - 1):
        label_sequence.append(label_matrix[i + seq_len])
    return np.array(label_sequence)


In [199]:
def pandas_fill(fea_matrix):
    df = pd.DataFrame(fea_matrix)
    out = df.fillna(method='ffill', inplace=False).values
    return out

In [206]:
fea_matrix = pm25_matrix.copy()

In [207]:
fea_matrix = pandas_fill(fea_matrix) 

In [208]:
fea_sequence = get_feature_sequence(fea_matrix)


In [202]:
masking_sequence = get_feature_sequence(masking)
delta_sequence = get_feature_sequence(delta)

In [210]:
pm25_label_sequence = get_label_sequence(pm25_matrix)

In [226]:
train_valid_test_split = np.array([len(train_index), len(validation_index), len(test_index)])

In [216]:
pm25_label_sequence.shape

(2861, 1)

In [229]:
fea_sequence.shape

(2861, 7, 1)

In [203]:
masking_sequence.shape

(2861, 7, 1)

In [204]:
delta_sequence.shape

(2861, 7, 1)

In [238]:
save_files_dict = {
    'train_valid_test_split.npy':train_valid_test_split,
    'fea_sequence.npy':fea_sequence,
    'masking_sequence.npy':masking_sequence,
    'delta_sequence.npy':delta_sequence,
    'pm25_label_sequence.npy':pm25_label_sequence
}

In [239]:
save_folder_path = 'data/raw'

In [240]:
for key in save_files_dict.keys():
    np.save(os.path.join(save_folder_path, key), save_files_dict[key])
